***
## Обработка параметров запроса в шаблоне

Получить доступ к параметрам запроса можно из HTML-шаблона или из view-функции: эти параметры хранятся в виде словаря в атрибуте `GET` объекта запроса `request`. Обратиться к этому словарю можно через `request.GET`. 

Заглянем в этот словарь: распечатаем его из view-функции `birthday()`.

```py
def birthday(request):
    print(request.GET)  # Напечатаем.
    form = BirthdayForm()
    context = {'form': form}
    return render(request, 'birthday/birthday.html', context) 
```

Отправьте любые данные из формы. В терминале появятся данные из запроса:

```
<QueryDict: {'first_name': ['Александр'], 'last_name': ['Пушкин'], 'birthday': ['1799-06-06']}> 
```

Параметры запроса можно получить и в шаблоне: для этого точно так же нужно обратиться к объекту `{{ request.GET }}`. Получив эти параметры, можно вывести их на страницу, например — поприветствовать пользователя:

```html
<!-- templates/birthday/birthday.html -->
{% extends "base.html" %}

{% block content %}
  <form>
    {{ form.as_p }}
    <input type="submit" value="Submit">
  </form>
  <!-- Привет, мы тебя знаем! -->
  <h2>Привет, {{ request.GET.first_name }} {{ request.GET.last_name }}</h2>
{% endblock %}
```

Шаблон *birthday.html* вызывается в двух случаях: 

* когда пользователь запрашивает страницу с формой,

* после того, как пользователь отправил форму.

В первом случае показывать блок с приветствием не следует: ведь никакой информации о пользователе у нас ещё нет, форма не отправлена.

Будем показывать блок с приветствием только в том случае, если в запросе есть GET-параметры, а проверку проведём прямо в шаблоне:

```html
<!-- templates/birthday/birthday.html -->
{% extends "base.html" %}

{% block content %}
  <form>
    {{ form.as_p }}
    <input type="submit" value="Submit">
  </form>
  <!-- Если в запросе есть параметры... -->
  {% if request.GET %}
    <!-- ...поприветствуем пользователя: -->
    <h2>Привет, {{ request.GET.first_name }} {{ request.GET.last_name }}</h2>
  {% endif %}
{% endblock %} 
```

Улучшим код шаблона: значение атрибута `request.GET` присвоим переменной `data`, это немного сократит код и позволит убрать повторные вызовы атрибута `GET` объекта `request`. 

Делается это с помощью парного тега `{% with ... %}`. 

```html
<!-- Либо так: -->
{% with data=request.GET %}
  ...
{% endwith %}
<!-- Либо так 
  (это старый синтаксис, но он тоже поддерживается): -->
{% with request.GET as data %}
  ...
{% endwith %} 
```

Добавим эту конструкцию в шаблон:

```html
{% extends "base.html" %}

{% block content %}
  <form>
    {{ form.as_p }}
    <input type="submit" value="Submit">
  </form>
  <!-- Присваиваем переменной data объект request.GET -->
  {% with data=request.GET %}
    <!-- В дальнейшем используем имя data вместо request.GET -->
    {% if data %}
      <h2>Привет, {{ data.first_name }} {{ data.last_name }}</h2>
    {% endif %}
  <!-- Когда заканчивается область шаблона, где нужна переменная, 
    закрываем тег with -->
  {% endwith %}
{% endblock %} 
```

***
## Валидация данных, полученных из веб-формы

В параметрах запроса могут быть переданы любые данные (ведь никто не мешает составить GET-запрос прямо в строке браузера); прежде чем вычислять количество дней, оставшихся до дня рождения, надо убедиться, что в запросе поступила именно дата, а не произвольная строка.

В классе `Form` есть встроенный валидатор, он проверяет переданные в форму данные на соответствие ожидаемым типам. 

Для валидирования данных

* параметры из запроса передают в форму:

```py
form = BirthdayForm(request.GET) 
```

* затем вызывают метод `form.is_valid()`: он возвращает `True`, если данные из запроса соответствуют ожидаемым типам в форме.

При успешной валидации значения полей, полученные в запросе, передаются в специальный словарь `form.cleaned_data`; ключи словаря совпадают с названиями полей, а значения приведены к нужным типам данных. 

Таким образом возникает два набора данных — два словаря: 

* `request.GET` — невалидированные данные из запроса; все значения — в виде строк;

* `form.cleaned_data` — валидированные данные; все значения приведены к ожидаемым типам данных — к тем типам, которые указаны для каждого поля в классе формы.

Для дальнейшей обработки данных, полученных в запросе, нужно брать значения из словаря `cleaned_data` — данные в этом словаре валидны и соответствуют ожидаемому типу.

Дополните view-функцию `birthday()`:

```py
def birthday(request):
    # Если есть параметры GET-запроса...
    if request.GET:
        # ...передаём параметры запроса в конструктор класса формы.
        form = BirthdayForm(request.GET)
        # Если данные валидны...
        if form.is_valid():
            # ...то считаем, сколько дней осталось до дня рождения.
            # Пока функции для подсчёта дней нет — поставим pass:
            pass
    # Если нет параметров GET-запроса.
    else:
        # То просто создаём пустую форму.
        form = BirthdayForm()
    # Передаём форму в словарь контекста:
    context = {'form': form}
    return render(request, 'birthday/birthday.html', context) 
```

Немного упростим код view-функции с помощью ловкого трюка:

```py
def birthday(request):
    form = BirthdayForm(request.GET or None)
    if form.is_valid():
        pass
    context = {'form': form}
    return render(request, 'birthday/birthday.html', context) 
```

Этот код гораздо короче, а работает точно так же, как и предыдущий вариант. Весь фокус в выражении `BirthdayForm(request.GET or None)`. 

Его логика такова: если в GET-запросе были переданы параметры — значит, объект `request.GET` не пуст и этот объект передаётся в форму; если же объект `request.GET` пуст — срабатывает условиe `or` и форма создаётся без параметров, через `BirthdayForm(None)` — это идентично обычному `BirthdayForm()`.

Таким образом в одном выражении совмещены две ветки `if` из предыдущего листинга.

Обновите страницу в браузере, заполните форму и отправьте её.

***
## Проверка данных на валидность

Составим и отправим невалидный GET-запрос: http://127.0.0.1:8000/birthday/?first_name=&last_name=Пушкин&birthday=чудноеМгновенье

Имена полей — те же, что и в форме, но почти все требования нарушены: вместо даты — строка, поле для имени — пустое.

На странице появятся сообщения об ошибках, но страница всё равно поприветствует пользователя:

![alt text](https://pictures.s3.yandex.net/resources/144_1682596038.png)

Такое поведение выглядит не очень логичным. Модифицируем шаблон так, чтобы приветствие выводилось не просто при наличии GET-параметров, а только при условии, что форма валидна. 

Замените в шаблоне условие `{% if data %}` на `{% if form.is_valid %}`:

```html
{% with data=request.GET %}
  <!-- Поменяйте условие тут -->
  {% if form.is_valid %}
    <h2>Привет, {{ data.first_name }} {{ data.last_name}}</h2>
  {% endif %}
{% endwith %} 
```

Теперь приветствие будет появляться только при полностью валидных данных.

Проверка `is_valid` выполняется и в шаблоне, и во view-функции. В шаблоне проверка предотвращает вывод приветствия, если форма содержит ошибки. Во view-функции проверка обеспечивает валидность данных, которые будут переданы для дальнейшей обработки. 

Обе эти проверки независимы друг от друга и не могут быть заменены одна на другую.

***
## Сколько ждать дня рождения?

Если до дня рождения осталось 0 дней — значит, сегодня праздник, на страницу надо вывести поздравление. Во всех остальных случаях нужно вывести сообщение «Осталось дней до дня рождения: N». Переменную, в которой будет храниться результат вычислений, назовём `birthday_countdown`. 

Первым делом вставим в шаблон условие, которое будет проверять значение переменной `{{ birthday_countdown }}`:

```html
{% with data=request.GET %}
  {% if form.is_valid %}
    <h2>Привет, {{ data.first_name }} {{ data.last_name }}</h2>
    <!-- Если до дня рождения 0 дней... -->
    {% if birthday_countdown == 0 %}
      <!-- ...выводим поздравление: -->
      С днём рождения!
    <!-- Если до дня рождения не ноль дней... -->
    {% else %}
      <!-- ...сообщаем, сколько осталось ждать: -->
      Осталось дней до дня рождения: {{ birthday_countdown }}!
    {% endif %}
  {% endif %}
{% endwith %} 
```

Многие логические и математические вычисления можно выполнить как в Python, так и в шаблонах. В большинстве случаев логику работы приложения («бизнес-логику») описывают в Python-коде, выносить её в шаблоны не принято; однако какие-то простые условия, как в нашем примере, можно описать и в шаблоне.

Осталось вычислить количество дней до дня рождения. Эти вычисления не относятся напрямую к теме форм, но нельзя же бросать проект на половине пути. 

Для вычислений создадим отдельную функцию, а потом вызовем её из view-функции; пояснения к коду есть в комментариях.

Создайте файл *birthday/utils.py* и перенесите в него код из листинга:

```py
# birthday/utils.py 
# Импортируем модуль для работы с датами.
from datetime import date

def calculate_birthday_countdown(birthday):
    """
    Возвращает количество дней до следующего дня рождения.

    Если день рождения сегодня, то возвращает 0.
    """
    # Сохраняем текущую дату в переменную today.
    today = date.today()
    # Получаем день рождения в этом году
    # с помощью вспомогательной функции (ниже).
    this_year_birthday = get_birthday_for_year(birthday, today.year)

    # Если день рождения уже прошёл...
    if this_year_birthday < today:
        # ...то следующий ДР будет в следующем году.
        next_birthday = get_birthday_for_year(birthday, today.year + 1)
    else:
        # А если в этом году ещё не было ДР, то он и будет следующим.
        next_birthday = this_year_birthday

    # Считаем разницу между следующим днём рождения и сегодняшним днём в днях.
    birthday_countdown = (next_birthday - today).days
    return birthday_countdown

def get_birthday_for_year(birthday, year):
    """
    Получает дату дня рождения для конкретного года.

    Ошибка ValueError возможна только в случае
    с високосными годами и ДР 29 февраля.
    В этом случае приравниваем дату ДР к 1 марта.
    """
    try:
        # Пробуем заменить год в дате рождения на переданный в функцию.
        calculated_birthday = birthday.replace(year=year)
    # Если возникла ошибка, значит, день рождения 29 февраля
    # и подставляемый год не является високосным.
    except ValueError:
        # В этом случае устанавливаем ДР 1 марта.
        calculated_birthday = date(year=year, month=3, day=1)
    return calculated_birthday 
```

Теперь вызовем функцию `calculate_birthday_countdown()` из view-функции `birthday()`. 

Чтобы передать в функцию `calculate_birthday_countdown()` дату рождения именно как дату (а не как строку), значение даты нужно взять из словаря `form.cleaned_data`, в котором содержатся валидированные данные, приведённые к нужным типам. 

Дата дня рождения в этом словаре доступна через выражение `form.cleaned_data['birthday']`; тип данных у этого объекта — `datetime.date`, а не просто строка. То, что надо.

```py
# birthday/views.py 
from django.shortcuts import render

from .forms import BirthdayForm
# Импортируем из utils.py функцию для подсчёта дней.
from .utils import calculate_birthday_countdown

def birthday(request):
    form = BirthdayForm(request.GET or None)
    # Создаём словарь контекста сразу после инициализации формы.
    context = {'form': form}
    # Если форма валидна...
    if form.is_valid():
        # ...вызовем функцию подсчёта дней:
        birthday_countdown = calculate_birthday_countdown(
            # ...и передаём в неё дату из словаря cleaned_data.
            form.cleaned_data['birthday']
        )
        # Обновляем словарь контекста: добавляем в него новый элемент.
        context.update({'birthday_countdown': birthday_countdown})
    return render(request, 'birthday/birthday.html', context) 
Обновите страницу и проверьте, как работает проект: передайте через форму сегодняшнюю дату и любую другую.Дата дня рождения в этом словаре доступна через выражение form.cleaned_data['birthday']; тип данных у этого объекта — datetime.date, а не просто строка. То, что надо.

# birthday/views.py 
from django.shortcuts import render

from .forms import BirthdayForm
# Импортируем из utils.py функцию для подсчёта дней.
from .utils import calculate_birthday_countdown

def birthday(request):
    form = BirthdayForm(request.GET or None)
    # Создаём словарь контекста сразу после инициализации формы.
    context = {'form': form}
    # Если форма валидна...
    if form.is_valid():
        # ...вызовем функцию подсчёта дней:
        birthday_countdown = calculate_birthday_countdown(
            # ...и передаём в неё дату из словаря cleaned_data.
            form.cleaned_data['birthday']
        )
        # Обновляем словарь контекста: добавляем в него новый элемент.
        context.update({'birthday_countdown': birthday_countdown})
    return render(request, 'birthday/birthday.html', context) 
```

Обновите страницу и проверьте, как работает проект: передайте через форму сегодняшнюю дату и любую другую.

![alt text](https://pictures.s3.yandex.net/resources/145_1682596082.png)